# Tasks for data preparation

- ✅ Aggregate movement data by hour
- ✅ Join together all datasets
- ❌ Aggregate data by day

## Table of contents
### 1. Load the data
### 2. Transform data
### 3. Join the data

## 1. Load the data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import plotly.express as px

In [141]:
# Load movements
movements2023 = pd.read_csv('../data/2023_verkehrszaehlungen_werte_fussgaenger_velo.csv')
# Rename columns
new_column_names = ['Standort', 'Datum', 'VELO_IN', 'VELO_OUT', 'FUSS_IN', 'FUSS_OUT', 'Ost', 'Nord']
movements2023.columns = new_column_names

# Load metadata
meta = pd.read_csv('../data/GeoData_StadtZurich/data/taz.view_eco_standorte.csv')

# Load weather data
weather2023 = pd.read_csv('../data/ugz_ogd_meteo_h1_2023.csv')

# Load population data
population = pd.read_csv('../data/bev324od3243.csv')

## 2. Overview of the data

### Display movements data

In [142]:
# Format Datum
movements2023['Datum'] = pd.to_datetime(movements2023['Datum'])
# Remove minutes information 
movements2023['Datum'] = movements2023['Datum'].dt.floor('1h')
# Format the timestamps and put time and date it into extra columns
movements2023['Date'] = movements2023['Datum'].dt.strftime('%Y-%m-%d')
movements2023['Time'] = movements2023['Datum'].dt.strftime('%H:%M')
movements2023['Datetime'] = movements2023['Datum'].dt.strftime('%Y-%m-%d %H:%M')

# Overview movements2023
movements2023.head()

,Standort,Datum,VELO_IN,VELO_OUT,FUSS_IN,FUSS_OUT,Ost,Nord,Date,Time,Datetime
0,5003,2023-01-01,NaN,NaN,1.0,0.0,2682978,1248744,2023-01-01,00:00,2023-01-01 00:00
1,4257,2023-01-01,0.0,0.0,NaN,NaN,2681857,1251991,2023-01-01,00:00,2023-01-01 00:00
2,394,2023-01-01,NaN,NaN,1.0,1.0,2683573,1251687,2023-01-01,00:00,2023-01-01 00:00
3,2986,2023-01-01,0.0,0.0,NaN,NaN,2684578,1251966,2023-01-01,00:00,2023-01-01 00:00
4,3598,2023-01-01,0.0,0.0,NaN,NaN,2684006,1246566,2023-01-01,00:00,2023-01-01 00:00


### Display meta data

In [161]:
# Remove columns that are not needed
meta.drop(['bis', 'fk_zaehler', 'von'], axis = 1)
meta.head()

,abkuerzung,bezeichnung,bis,fk_zaehler,id1,richtung_in,richtung_out,von,objectid,korrekturfaktor,geometry
0,VZS_BINZ,Binzmühlestrasse,2.018040e+13,ECO09113500,7,Oerlikon,Glaubtenstrasse,20101130000000,1,1.22,POINT (2681857 1251990.9)
1,VZS_BUCH,Bucheggplatz,2.019031e+13,ECO09113505,16,Hofwiesenstrasse,Höngg,20140626000000,2,1.27,POINT (2682647 1250363.9)
2,VZS_KLOW,Kloster-Fahr-Weg,2.020020e+13,Y0412032046,53,Stadtgrenze,Innenstadt,20100625000000,3,0.94,POINT (2679028 1250673.9)
3,VZS_HOFW,Hofwiesenstrasse,2.019031e+13,ECO09113507,13,Bucheggplatz,---,20100923000000,4,1.28,POINT (2682683.2 1250569.6)
4,VZS_LIMB,Limmatquai --> Bellevue,2.019060e+13,Y2G14045587,61,Bellevue,---,20140625000000,5,1.47,POINT (2683447 1247062.9)


### Display weather data
The weather data is available for three different locations `['Zch_Stampfenbachstrasse', 'Zch_Schimmelstrasse', 'Zch_Rosengartenstrasse']`. We will need only one of these.

In [144]:
# Filter only Stampfenbachstrasse
weather2023 = weather2023[weather2023['Standort'] == 'Zch_Stampfenbachstrasse']

# Remove coumns
weather2023.pop('Standort')
weather2023.pop('Status')

# Overview
weather2023.head()

,Datum,Parameter,Intervall,Einheit,Wert
0,2023-01-01T00:00+0100,T,h1,°C,11.57
1,2023-01-01T00:00+0100,Hr,h1,%Hr,72.29
2,2023-01-01T00:00+0100,p,h1,hPa,971.62
3,2023-01-01T00:00+0100,RainDur,h1,min,0.00
4,2023-01-01T00:00+0100,StrGlo,h1,W/m2,0.01


### Display population data

In [145]:
population.tail()

,StichtagDatJahr,AnzBestWir
118,2019,434008
119,2020,434736
120,2021,436332
121,2022,443037
122,2023,447082


## 2. Transform data

### Long to wide transformation for weather data

In [146]:
# Transform weather data into wide format
weather2023['Param_Unit'] = weather2023['Parameter'] + ' [' + weather2023['Einheit'] + ']'
wide_weather2023 = weather2023.pivot_table(index=['Datum'], columns='Param_Unit', values='Wert').reset_index()

# Display the wide format DataFrame
wide_weather2023.head()

Param_Unit,Datum,Hr [%Hr],RainDur [min],StrGlo [W/m2],T [°C],WD [°],WVs [m/s],WVv [m/s],p [hPa]
0,2023-01-01T00:00+0100,72.29,0.0,0.01,11.57,169.49,1.95,1.69,971.62
1,2023-01-01T01:00+0100,63.66,0.0,0.02,13.47,205.89,3.40,2.77,971.86
2,2023-01-01T02:00+0100,68.85,0.0,0.02,12.39,149.11,1.98,1.49,971.76
3,2023-01-01T03:00+0100,70.72,0.0,0.02,11.69,157.08,1.79,1.46,972.01
4,2023-01-01T04:00+0100,70.45,0.0,0.02,11.55,178.54,2.98,2.74,972.10


In [147]:
# Format Datum
wide_weather2023['Datum'] = pd.to_datetime(wide_weather2023['Datum'])
# Remove minutes information 
wide_weather2023['Datum'] = wide_weather2023['Datum'].dt.floor('1h')
# Format the timestamps and put time and date it into extra columns
wide_weather2023['Year'] = wide_weather2023['Datum'].dt.year
wide_weather2023['Date'] = wide_weather2023['Datum'].dt.strftime('%Y-%m-%d')
wide_weather2023['Time'] = wide_weather2023['Datum'].dt.strftime('%H:%M')
wide_weather2023['Datetime'] = wide_weather2023['Datum'].dt.strftime('%Y-%m-%d %H:%M')

# Overview
wide_weather2023.head()

Param_Unit,Datum,Hr [%Hr],RainDur [min],StrGlo [W/m2],T [°C],WD [°],WVs [m/s],WVv [m/s],p [hPa],Year,Date,Time,Datetime
0,2023-01-01 00:00:00+01:00,72.29,0.0,0.01,11.57,169.49,1.95,1.69,971.62,2023,2023-01-01,00:00,2023-01-01 00:00
1,2023-01-01 01:00:00+01:00,63.66,0.0,0.02,13.47,205.89,3.40,2.77,971.86,2023,2023-01-01,01:00,2023-01-01 01:00
2,2023-01-01 02:00:00+01:00,68.85,0.0,0.02,12.39,149.11,1.98,1.49,971.76,2023,2023-01-01,02:00,2023-01-01 02:00
3,2023-01-01 03:00:00+01:00,70.72,0.0,0.02,11.69,157.08,1.79,1.46,972.01,2023,2023-01-01,03:00,2023-01-01 03:00
4,2023-01-01 04:00:00+01:00,70.45,0.0,0.02,11.55,178.54,2.98,2.74,972.10,2023,2023-01-01,04:00,2023-01-01 04:00


### Aggregate movement data per hour

In [148]:
movements2023.head()
# Aggregate based on 'Standort' and 'Time', sum 'VELO_IN', and get max 'Ost'
movements2023_hourly = movements2023.groupby(['Standort', 'Date', 'Time', 'Datetime']).agg({'VELO_IN' : 'sum',
                                                                        'VELO_OUT' : 'sum',
                                                                        'FUSS_IN' : 'sum',
                                                                        'FUSS_OUT' : 'sum',
                                                                        'Ost' : 'max',
                                                                        'Nord' : 'max'})

In [149]:
movements2023_hourly.head(25)

VELO_IN  VELO_OUT  FUSS_IN  \
Standort Date       Time  Datetime                                       
20       2023-01-01 00:00 2023-01-01 00:00      0.0       0.0     46.0   
                    01:00 2023-01-01 01:00      0.0       0.0     43.0   
                    02:00 2023-01-01 02:00      0.0       0.0     36.0   
                    03:00 2023-01-01 03:00      0.0       0.0     22.0   
                    04:00 2023-01-01 04:00      0.0       0.0     11.0   
                    05:00 2023-01-01 05:00      0.0       0.0     18.0   
                    06:00 2023-01-01 06:00      0.0       0.0     19.0   
                    07:00 2023-01-01 07:00      0.0       0.0      7.0   
                    08:00 2023-01-01 08:00      0.0       0.0      4.0   
                    09:00 2023-01-01 09:00      0.0       0.0     14.0   
                    10:00 2023-01-01 10:00      0.0       0.0     20.0   
                    11:00 2023-01-01 11:00      0.0       0.0     26.0   
                    12:00 2023-01-01 12:00      0.0       0.0     32.0   
                    13:00 2023-01-01 13:00      0.0       0.0     43.0   
                    14:00 2023-01-01 14:00      0.0       0.0     41.0   
                    15:00 2023-01-01 15:00      0.0       0.0     67.0   
                    16:00 2023-01-01 16:00      0.0       0.0     46.0   
                    17:00 2023-01-01 17:00      0.0       0.0     51.0   
                    18:00 2023-01-01 18:00      0.0       0.0     32.0   
                    19:00 2023-01-01 19:00      0.0       0.0     19.0   
                    20:00 2023-01-01 20:00      0.0       0.0     15.0   
                    21:00 2023-01-01 21:00      0.0       0.0      7.0   
                    22:00 2023-01-01 22:00      0.0       0.0     11.0   
                    23:00 2023-01-01 23:00      0.0       0.0      4.0   
         2023-01-02 00:00 2023-01-02 00:00      0.0       0.0      2.0   

                                            FUSS_OUT      Ost     Nord  
Standort Date       Time  Datetime                                      
20       2023-01-01 00:00 2023-01-01 00:00      31.0  2682689  1247735  
                    01:00 2023-01-01 01:00      94.0  2682689  1247735  
                    02:00 2023-01-01 02:00      27.0  2682689  1247735  
                    03:00 2023-01-01 03:00      27.0  2682689  1247735  
                    04:00 2023-01-01 04:00      33.0  2682689  1247735  
                    05:00 2023-01-01 05:00      14.0  2682689  1247735  
                    06:00 2023-01-01 06:00       6.0  2682689  1247735  
                    07:00 2023-01-01 07:00       0.0  2682689  1247735  
                    08:00 2023-01-01 08:00       6.0  2682689  1247735  
                    09:00 2023-01-01 09:00       1.0  2682689  1247735  
                    10:00 2023-01-01 10:00       6.0  2682689  1247735  
                    11:00 2023-01-01 11:00      10.0  2682689  1247735  
                    12:00 2023-01-01 12:00      18.0  2682689  1247735  
                    13:00 2023-01-01 13:00      28.0  2682689  1247735  
                    14:00 2023-01-01 14:00      27.0  2682689  1247735  
                    15:00 2023-01-01 15:00      47.0  2682689  1247735  
                    16:00 2023-01-01 16:00      51.0  2682689  1247735  
                    17:00 2023-01-01 17:00      24.0  2682689  1247735  
                    18:00 2023-01-01 18:00      27.0  2682689  1247735  
                    19:00 2023-01-01 19:00      23.0  2682689  1247735  
                    20:00 2023-01-01 20:00      14.0  2682689  1247735  
                    21:00 2023-01-01 21:00       9.0  2682689  1247735  
                    22:00 2023-01-01 22:00      21.0  2682689  1247735  
                    23:00 2023-01-01 23:00      19.0  2682689  1247735  
         2023-01-02 00:00 2023-01-02 00:00       8.0  2682689  1247735

## 3. Join the data

In [158]:
# Join movements and weather
df_agg_hourly = pd.merge(movements2023_hourly, wide_weather2023.drop('Datum', axis = 1), 
         left_on = 'Datetime', right_on = 'Datetime', how = 'left')

# Join population data
df_agg_hourly = pd.merge(df_agg_hourly, population,
                         left_on = 'Year', right_on = 'StichtagDatJahr', how = 'left')

# Remove 'StichtagDatJahr'
df_agg_hourly = df_agg_hourly.drop('StichtagDatJahr', axis = 1)

# Join meta data
df_agg_hourly = pd.merge()

# Display
df_agg_hourly.head()

,Datetime,VELO_IN,VELO_OUT,FUSS_IN,FUSS_OUT,Ost,Nord,Hr [%Hr],RainDur [min],StrGlo [W/m2],T [°C],WD [°],WVs [m/s],WVv [m/s],p [hPa],Year,Date,Time,AnzBestWir
0,2023-01-01 00:00,0.0,0.0,46.0,31.0,2682689,1247735,72.29,0.0,0.01,11.57,169.49,1.95,1.69,971.62,2023.0,2023-01-01,00:00,447082.0
1,2023-01-01 01:00,0.0,0.0,43.0,94.0,2682689,1247735,63.66,0.0,0.02,13.47,205.89,3.40,2.77,971.86,2023.0,2023-01-01,01:00,447082.0
2,2023-01-01 02:00,0.0,0.0,36.0,27.0,2682689,1247735,68.85,0.0,0.02,12.39,149.11,1.98,1.49,971.76,2023.0,2023-01-01,02:00,447082.0
3,2023-01-01 03:00,0.0,0.0,22.0,27.0,2682689,1247735,70.72,0.0,0.02,11.69,157.08,1.79,1.46,972.01,2023.0,2023-01-01,03:00,447082.0
4,2023-01-01 04:00,0.0,0.0,11.0,33.0,2682689,1247735,70.45,0.0,0.02,11.55,178.54,2.98,2.74,972.10,2023.0,2023-01-01,04:00,447082.0


# 4. Export the data

In [159]:
# Export as csv
df_agg_hourly.to_csv('../results/df_agg_hourly.csv', index=False)